In [1]:
try:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
except:
    pass

import cv2
import numpy as np
import matplotlib.pyplot as plt

cap = cv2.VideoCapture("/home/abhinav/Desktop/ENPM673/Project2/data/challenge_video.mp4")
K = np.array([[  1.15422732e+03,0.00000000e+00,6.71627794e+02],
              [  0.00000000e+00,1.14818221e+03,3.86046312e+02],
              [  0.00000000e+00,0.00000000e+00,1.00000000e+00]])
dist = np.array([ -2.42565104e-01,-4.77893070e-02,  -1.31388084e-03,  -8.79107779e-05,
    2.20573263e-02])
def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    res = cv2.LUT(image, lookUpTable)
    return res
def preprocessing(frame):
    th = np.median(frame[int(frame.shape[0]/2):,:,:])
#     frame = adjust_gamma(frame,gamma=1.5)
#     undist = cv2.undistort(frame,K,dist)
#     blur = cv2.bilateralFilter(undist,9,75,75)
#     median = cv2.medianBlur(blur,5)
    
    
#     gray = cv2.cvtColor(median, cv2.COLOR_BGR2HLS)
#     s_channel = gray[:,:,2]
#     l_channel = gray[:,:,1]
    
    #Create mask for S Channel
    if th<60:
        s_channel = adjust_gamma(frame,gamma=25.0)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_RGB2LAB)
        s_channel = gray[:,:,2]
        l_channel = gray[:,:,0]
        s_min = 135
        s_max = 250
        
        l_min = 200
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel<=s_max)]=1
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    
    elif th>130:
        s_channel = adjust_gamma(frame,gamma=1)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_BGR2LAB)
        s_channel = gray[:,:,2]
        l_channel = gray[:,:,0]
        s_min = 150
        s_max = 250
        
        l_min = 220
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel[1]<=s_max)]=1
        
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    else:
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_BGR2LAB)
        s_channel = gray[:,:,2]
        l_channel = gray[:,:,0]
        s_min = 150
        s_max = 250
        
        l_min = 190
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel[1]<=s_max)]=1
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    
    combined = np.zeros([frame.shape[0],frame.shape[1]])
    combined[(mask1==1)|(mask2==1)]=1
    mask = np.uint8(255*combined/np.max(combined))
#     mask = cv2.inRange(median,np.array([0,190,80]),np.array([255,255,150]))
    seg = cv2.bitwise_and(frame,median,mask=mask)
    return seg,combined,th
while(True):
    ret,frame = cap.read()

    seg,combined,th = preprocessing(frame)
    #Extract the region of interest and warp to get bird's eye view
    h = frame.shape[0]
    w = frame.shape[1]
#     src = np.array([[w/2-50,h/2+110],[w/2+50,h/2+110],[w/2+400,h/2],[w/2-400,h/2]])
#     dst = np.array([[0,0],[w,0],[w,h],[0,h]])
    src = np.array([[375,480],[905,480],[1811,685],[-531,685]])
    dst = np.array([[300,0],[800,0],[800,600],[300,600]])
#     src = np.array([[600,500],[770,500],[1050,680],[350,680]])
#     dst = np.array([[300,300],[700,300],[700,600],[300,600]])
    H,flag = cv2.findHomography(src,dst)
    out= cv2.warpPerspective(seg,H,(w,h))
    out_gray = cv2.cvtColor(out,cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(out_gray,100,255,cv2.THRESH_BINARY)

#     font = cv2.FONT_HERSHEY_SIMPLEX
#     cv2.putText(out,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
# #     cv2.putText(l_channel,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
#     cv2.putText(combined,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
    ###########################################
    #Histogram
    histogram = np.sum(thresh1[thresh1.shape[0]/2:,:],axis = 0)
    plt.plot(histogram)
    mid = np.int((dst[0][0]+dst[1][0])/2)
#     mid = np.int(histogram.shape[0]/2)
    left_lane = np.argmax(histogram[:mid])
    right_lane = np.argmax(histogram[mid:])+mid
    
    wndw = 50
    wndw_ht = np.int(thresh1.shape[0]/wndw)
    
    nonzero = thresh1.nonzero()
    x = np.array(nonzero[0])
    y = np.array(nonzero[1])
    
    left_curr = left_lane
    right_curr = right_lane
    
    margin = 20
    
    minPix = 10
    
    #Find indices for left and right pixels
    left_inds = []
    right_inds = []
    
    for win in range(wndw):
        y_min = thresh1.shape[0] - (win+1)*wndw_ht
        y_max = thresh1.shape[0] - (win)*wndw_ht
        x_min_left = left_curr - margin
        x_max_left = left_curr + margin
        x_min_right = right_curr - margin
        x_max_right = right_curr + margin
        
        cv2.rectangle(out,(x_min_left,y_min),(x_max_left,y_max),(255,0,255),2)
        cv2.rectangle(out,(x_min_right,y_min),(x_max_right,y_max),(255,0,0),2)
        
        left_ind = ((y>=y_min)&(y<y_max)&(x>=x_min_left)&(x<x_max_left)).nonzero()[0]
        right_ind = ((y>=y_min)&(y<y_max)&(x>=x_min_right)&(x<x_max_right)).nonzero()[0]
        
        left_inds.append(left_ind)
        right_inds.append(right_ind)
        #If no. of white pixels > minPix then start 
        #the new window form their mean position
        if len
        


    
    cv2.imshow('s',out)
    cv2.imshow('l',thresh1)
#     cv2.imshow('test',l_channel)
#     cv2.imshow('mask',mask)
    if cv2.waitKey(1)& 0xff==ord('q'):
        cv2.destroyAllWindows()
        break

AttributeError: 'NoneType' object has no attribute 'shape'

thresh1.nonzero()

In [46]:
mid

550

In [17]:
np.argmax(histogram[:600])

432

In [18]:
np.argmax(histogram[600:])

271

In [20]:
a = np.array([1,2,3,4,5,6,7,8,9])

In [23]:
np.argmax(a[5:])

3